<a href="https://colab.research.google.com/github/lahirunie-dulsara/EN3150-Assignment-3-CNN/blob/Sakith/RealWaste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile, os, re, shutil
from PIL import Image # Import the Image module from PIL

# === 2️⃣ Path to your ZIP file on Google Drive ===
# 👉 Change this path to match your actual file location
zip_path = "/content/drive/MyDrive/"

# === 3️⃣ Create folder to extract contents ===
extract_path = "/content/extracted_zip"
os.makedirs(extract_path, exist_ok=True)

# === 4️⃣ Unzip the folder ===
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("✅ Unzipped to:", extract_path)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
